In [1]:
from os.path import join
from glob import glob
import dask
import dask.array as da
import pyklb as klb
import napari
import time

# 2701 timepoints
#Shape of each 3D stack is (75,768,1848)

D:\Anaconda3\envs\denoising\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
D:\Anaconda3\envs\denoising\lib\site-packages\napari\__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
#Use opportunistic caching 
from dask.cache import Cache
cache = Cache(32e9)  # Set aside some memory
cache.register()    # Turn cache on globally

In [3]:
inPath = 'W:/SV3/RC_15-06-11/Dme_E2_His2AvRFP_spiderGFP_12-03_20150611_155054.corrected/Results/TimeFused/'
fnames = glob(join(inPath, '*/*CM01_CHN01.fusedStack.klb' ))
fnames.sort()
#fnames = fnames[0:1200]

t0 = time.time()
sample_full = klb.readfull(fnames[0]) #Sample image with klb.readfull
print('klb.readfull of a 3D stack (203 MB umcompressed) took', round(time.time()-t0,2), 'sec')

z= 64 #sample_full.shape[0]
y= sample_full.shape[1]
x= sample_full.shape[2]

#z-planes to chunk; select a divisor of z, otherwise pads extra z-planes
zstride= 8

klb.readfull of a 3D stack (203 MB umcompressed) took 0.99 sec


### Generate lazy arrays

In [4]:
# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_reader = dask.delayed(klb.readroi)  # lazy klbroi reader
lazy_values = [[lazy_reader(fn,[i,0,0],[i+zstride,y-1,x-1]) for fn in fnames] for i in range(0,z,zstride)] #list of lists

lazy_arrays = [da.from_delayed(lazy_value,
                               shape=(zstride,y,x),
                               dtype=sample_full.dtype)
               for lazy_value in lazy_values[0]]

#Stack all timepoints for the first z-plane into a dask array
z0_tstack = da.stack(lazy_arrays, axis=0)

#Visual of the dask array
z0_tstack

,Array,Chunk
Bytes,61.33 GB,22.71 MB
Shape,"(2701, 8, 768, 1848)","(1, 8, 768, 1848)"
Count,8103 Tasks,2701 Chunks
Type,uint16,numpy.ndarray


In [5]:
stacks = z0_tstack
for zblock in range(len(lazy_values)-1):
    lazy_arrays = [da.from_delayed(lazy_value,
                               shape=(zstride,y,x),
                               dtype=sample_full.dtype)
               for lazy_value in lazy_values[zblock+1]]
    #Stack all timepoints for a single z-plane; dimension of dask array: [t,1,y,x]
    tstack = da.stack(lazy_arrays, axis=0) 
    
    #Concatenate all timepoints into a new axis; dimension of dask array: [t,z,y,x]
    stacks = da.concatenate([stacks, tstack], axis=1)

#Visual of the dask array
stacks

,Array,Chunk
Bytes,490.68 GB,22.71 MB
Shape,"(2701, 64, 768, 1848)","(1, 8, 768, 1848)"
Count,159359 Tasks,21608 Chunks
Type,uint16,numpy.ndarray


###  Initialize the GUI now

In [6]:
%gui qt5

### Create a napari viewer and add the image as a layer

In [7]:
viewer = napari.Viewer(axis_labels = ['t','z','y','x'])
viewer.add_image(stacks,
                 scale = [1,8,1,1],
                 colormap='inferno',
                 blending='additive',
                 name='nuclei',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,180])

<Image layer 'nuclei' at 0x217d1c8f908>

### Miscellaneous

### Observations


Note: 16 GB of cache was allocated for these tests. The interactivity of the napari browser is smooth in 'z' and reasonably good in 't', under these scenarios:
- 100 timepoints (21 GB), z-stride of 8 (23 MB chunks)

Even this is better in both 'z' and 't' compared to 'load_klbs_dask_napari' which uses klb.readfull to read the 3D stacks, then rechunks the data into (1,8,768,1848) 
- 1200 timepoints (245 GB), z-stride of 8 (23 MB chunks)